In [1]:
################# User Inputs
# Paths
imDir = 'X:\\Force Project\\PublicationData\\SpinningDisk_Myosin\\3D_timelapse'
saveDir = 'X:\\Force Project\\PublicationData\\SpinningDisk_Myosin\\PublicationFigures\\MovieS2'
# Metadata
xyscale = 0.065 # um/pixel
zscale = 0.2 # um/pixel
tscale = 45/60 # minutes/frame


################# Set up
# Notebook
import tifffile as tf
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib
import colorcet as cc
from scipy.ndimage import center_of_mass
from skimage import measure

# Convert paths
imDir = Path(imDir)
saveDir = Path(saveDir)

# Consistent visualization parameters
gap = 6
gapz = np.round(gap/2).astype(int) # Roughly account for anistropy in the visualization

dataName = '20241015_U2OS_SGRLC_1to3_100Xoil_45mintimelapse' #Fig2D
zSlice =  3
# Cropping 
x1 = 225
x2 = 2000
y1 = 420
y2 = 1820
ySlice = 1500 - y1 + 1

# Set up the paths
dataDir = str(imDir) + '\\OpticalFlow3D_Python\\' + dataName
dataDir = Path(dataDir)

# Use a consistent colorscale across different figures
cmin = 0
cmax = 180
colormap = cc.m_CET_D1_r

In [4]:
# Contractility over time with the image size by size
imName = dataName + '.tif'
allImages = tf.memmap(imDir / imName) # Does not load images into memory until called later
contrast = [80,800] # Determined empirically

for f in np.arange(3,58):

    im = allImages[f,:]
    im = im[zSlice,y1:y2,x1:x2]
    im = im.astype(float)
    im = np.clip(im,contrast[0],contrast[1])
    im = (im-contrast[0])/(contrast[1]-contrast[0])

    fig, ax = plt.subplots(1,2,sharex='col',layout="constrained")
    fig.set_figwidth(20)
    fig.set_figheight(10)

    # Intensity Image
    ax[0].imshow(im,cmap='Grays_r')
    ax[0].plot([50, 50+25/xyscale],[1350, 1350],color='white',linewidth=8)

    tNow = f*tscale
    mNow = np.floor(tNow)
    sNow = (tNow-mNow)*60
    tNow = str(int(mNow)).zfill(2) + ':' + str(int(sNow)).zfill(2)

    ax[0].annotate(tNow,[20,20],color='w',size=20)

    ax[0].axis('off')
    fig.set_facecolor("black")

    # Reliability thresholding
    relFile = dataName + '_rel_t' + str(f).zfill(4) + '.tiff'
    rel = tf.imread(dataDir / relFile)
    relThresh = np.percentile(rel,90)
    relMask = rel > relThresh
    relMask = relMask[:,y1:y2,x1:x2] # Crop for clarity of visualization

    # Clean up relMask: Keep only the largest object (some cells have a small piece of another cell in the FOV)
    labels_mask = measure.label(relMask)                       
    regions = measure.regionprops(labels_mask)
    regions.sort(key=lambda x: x.area, reverse=True)
    if len(regions) > 1:
        for rg in regions[1:]:
            # print(rg.coords)
            labels_mask[rg.coords[:,0], rg.coords[:,1], rg.coords[:,2]] = 0
    labels_mask[labels_mask!=0] = 1
    relMask = labels_mask

    # Load and format velocity data
    vxFile = dataName + '_vx_t' + str(f).zfill(4) + '.tiff'
    vx = tf.imread(dataDir / vxFile)
    vx = vx[:,y1:y2,x1:x2] # Crop for clarity of visualization
    vx = vx*relMask # Remove unreliable vectors    
    vx[vx==0] = np.nan    

    vyFile = dataName + '_vy_t' + str(f).zfill(4) + '.tiff'
    vy = tf.imread(dataDir / vyFile)
    vy = vy[:,y1:y2,x1:x2] # Crop for clarity of visualization
    vy = vy*relMask # Remove unreliable vectors
    vy[vy==0] = np.nan    

    vzFile = dataName + '_vz_t' + str(f).zfill(4) + '.tiff'    
    vz = tf.imread(dataDir / vzFile)
    vz = vz[:,y1:y2,x1:x2] # Crop for clarity of visualization
    vz = vz*relMask # Remove unreliable vectors
    vz[vz==0] = np.nan

    # Put into physical units for remaining calculations
    vx = vx*xyscale/tscale 
    vy = vy*xyscale/tscale
    vz = vz*zscale/tscale

    # Set up a grid for quiver plots
    x = np.linspace(0, vx.shape[2]-1, vx.shape[2])*xyscale
    y = np.linspace(0, vx.shape[1]-1, vx.shape[1])*xyscale
    z = np.linspace(0, vx.shape[0]-1, vx.shape[0])*zscale
    X, Y, Z = np.meshgrid(x,y,z)
    X = np.moveaxis(X,-1,0)
    Y = np.moveaxis(Y,-1,0)
    Z = np.moveaxis(Z,-1,0)

    # Calculate useful quantities
    speed = np.sqrt(np.power(vx,2)+np.power(vy,2)+np.power(vz,2))

    # Describe contractility as the flow with respect to the cell centroid
    com = center_of_mass(relMask)
    # Direction to the centroid
    dxc = X-com[2]*xyscale
    dyc = Y-com[1]*xyscale
    dzc = Z-com[0]*zscale

    normalc = np.stack([-dxc, -dyc, -dzc], axis=-1)
    normc = np.linalg.norm(normalc, axis=-1, keepdims=True)

    normalc[:, :, :, 0] /= normc[:, :, :, 0]
    normalc[:, :, :, 1] /= normc[:, :, :, 0]
    normalc[:, :, :, 2] /= normc[:, :, :, 0]

    # Normalize the velocity vectors
    vnorm = np.stack([vx/speed, vy/speed, vz/speed], axis=-1)

    dotc = np.vecdot(vnorm,normalc)

    angle_radc = np.arccos(dotc)
    angle_degc = np.degrees(angle_radc)

    norm = matplotlib.colors.Normalize(vmin=cmin, vmax=cmax)
    speedPlot = norm(angle_degc[zSlice,::gap,::gap])
    speedPlot = speedPlot.flatten()    

    # Make the quiver plot
    q = ax[1].quiver(X[zSlice,::gap,::gap],Y[zSlice,::gap,::gap],vx[zSlice,::gap,::gap],vy[zSlice,::gap,::gap],
            color=colormap(speedPlot),
            scale=1/15,angles='xy',scale_units='xy',units='xy',headwidth=4)

    # Format the quiver plot
    ax[1].set_xlim(np.min(X),np.max(X))
    ax[1].set_ylim(np.max(Y),np.min(Y))
    ax[1].set_aspect('equal')
    ax[1].axis('off')
    fig.set_facecolor("black")

    # Save and show
    saveName = 'MovieS2_zSlice' + str(zSlice) + '_gap' + str(gap) + '_cmin' + str(cmin) + '_cmax' + str(cmax) + '_frame' + str(f).zfill(4) + '.png'
    plt.savefig(saveDir / saveName, dpi=300, bbox_inches='tight')
    plt.close(fig)
